In [ ]:
%tensorflow_version 1.x
import tensorflow.keras as K
import numpy as np
import matplotlib.pyplot as plt

TensorFlow 1.x selected.


In [ ]:
def preprocess_data(X, Y):
    X = K.applications.resnet50.preprocess_input(X)
    Y = K.utils.to_categorical(Y, 10)

    return X, Y

In [ ]:
if __name__ == '__main__':
    # Divide the data in Train and Test Datasets
    (x_train, y_train),(x_test, y_test) = K.datasets.cifar10.load_data()

    x_train, y_train = preprocess_data(x_train, y_train)
    x_test, y_test = preprocess_data(x_test, y_test)
    
    model = K.applications.ResNet50(include_top=False,
                                 weights='imagenet',
                                 input_shape=(224, 224, 3))

    model_1= K.Sequential()
    model_1.add(K.layers.UpSampling2D((7, 7)))
    model_1.add(model)
    model_1.add(K.layers.AveragePooling2D(pool_size=7)) 
    model_1.add(K.layers.Flatten())
    model_1.add(K.layers.Dense(10,activation=('softmax')))

    # model.summary()

    checkpoint = K.callbacks.ModelCheckpoint(filepath='cifar10.h5',
                                             monitor='val_acc',
                                             mode='max',
                                             verbose=1,
                                             save_best_only=True)
    
    earlys = K.callbacks.EarlyStopping(monitor='val_acc',
                                   mode='max',
                                   verbose=1,
                                   patience=10)

    model_1.compile(optimizer=K.optimizers.RMSprop(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['acc'])

    model_1.fit(x_train, y_train,
                validation_data=(x_test, y_test),
                batch_size=32,
                epochs=5,
                verbose=1,
                callbacks=[checkpoint, earlys])

    model_1.save('cifar10.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
49984/50000 [============================>.] - ETA: 0s - loss: 0.3641 - acc: 0.8794
Epoch 00001: val_acc improved from -inf to 0.90720, saving model to cifar10.h5
50000/50000 [==============================] - 1407s 28ms/sample - loss: 0.3641 - acc: 0.8794 - val_loss: 0.3925 - val_acc: 0.9072
Epoch 2/5
49984/50000 [============================>.] - ETA: 0s - loss: 0.1633 - acc: 0.9463
Epoch 00002: val_acc improved from 0.90720 to 0.92420, saving model to cifar10.h5
50000/50000 [==============================] - 1392s 28ms/sample - loss: 0.1634 - acc: 0.9463 - val_loss: 0.3236 - val_acc: 0.9242
Epoch 3/5
49984/50000 [============================>.] - ETA: 0s - loss: 0.1024 - acc: 0.9651
Epoch 00003: val_acc improved from 0.92420 to 0.92860, saving model to cifar10.h5
50000/50000 [==============================] - 1393s 28ms/sample - loss: 0.1024 - acc: 0.9651 - val_loss: 0.2611 - val_acc: 0.9286
Epoch 4/5
49984/50000 [=========

In [ ]:
# fix issue with saving keras applications
K.learning_phase = K.backend.learning_phase 

_, (X, Y) = K.datasets.cifar10.load_data()
X_p, Y_p = preprocess_data(X, Y)
model = K.models.load_model('cifar10.h5')
model.evaluate(X_p, Y_p, batch_size=128, verbose=1)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
10000/10000 [==============================] - 70s 7ms/sample - loss: 0.2773 - acc: 0.9366


[0.27729836688041687, 0.9366]

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d_2 (UpSampling2 multiple                  0         
_________________________________________________________________
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
average_pooling2d_2 (Average multiple                  0         
_________________________________________________________________
flatten_2 (Flatten)          multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  20490     
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________
